#Sublimation Enthalpy_CV

In [1]:
# Some necessary modules are loaded
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

In [2]:
# Reading data from the data file
df   = pd.read_csv('fp_cv.csv', delimiter=',', header=0)
data_nonan = np.array(df)


# X (fingerprint) and Y (property) of the polymers 
X_nonan = data_nonan[:,1:]
Y = data_nonan[:,0]

#Standardization MinMaxScaler 
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X_nonan)

print (np.shape(data_nonan))

(299, 145)


In [3]:
# Split the data into training and test sets
test_size = 0.1
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size, random_state=1)
print ("X_train",np.shape(X_train))
print ("X_test",np.shape(X_test))
print ("Y_train",np.shape(Y_train))
print ("Y_test",np.shape(Y_test))

X_train (269, 144)
X_test (30, 144)
Y_train (269,)
Y_test (30,)


In [4]:
# Some initial parameters to determine the hyperparameters 
Y_average = np.average(Y)
noise_avr = np.std(Y)
noise_lb  = noise_avr/10
noise_ub  = noise_avr*10
n_fold    = 5

# The prior of the GPR model
kernel   = (Y_average)**2*RBF(length_scale=1)+WhiteKernel(noise_level=noise_avr**2,noise_level_bounds=(noise_lb**2, noise_ub**2))
#gp       = GaussianProcessRegressor(kernel=kernel, alpha=0, n_restarts_optimizer=5)

# Now training the GPR model
#opt_gp   = gp
opt_rmse = 1.0E20
ncv      = 0
ncv_opt  = ncv

# Training set splitted into n_fold subsets
kf_      = KFold(n_splits=n_fold, shuffle = True)
kf       = kf_.split(X_train)

##print (kf_.get_n_splits(kf))
# Loop for the best kernal
for train, test in kf:
    ##print (train.shape)
    ##print (test.shape)
    X_cv_train = X_train[train]
    X_cv_test  = X_train[test]
    Y_cv_train = Y_train[train]
    Y_cv_test  = Y_train[test]
    
    gp         = GaussianProcessRegressor(kernel=kernel, alpha=0, n_restarts_optimizer=10)
    gp.fit(X_cv_train, Y_cv_train)
    y_cv_train = gp.predict(X_cv_train, return_std=False) 
    y_cv_test  = gp.predict(X_cv_test, return_std=False)  

    rmse_cv_train = np.sqrt(mean_squared_error(Y_cv_train, y_cv_train))
    rmse_cv_test  = np.sqrt(mean_squared_error(Y_cv_test, y_cv_test))
    print('        ncv, rmse_train, rmse_test: ', ncv, rmse_cv_train, rmse_cv_test)

    if rmse_cv_test < opt_rmse:
        opt_rmse = rmse_cv_test
        opt_gp   = gp #如果有满足条件的优化，就选优化后的kernal，否则还使用原始的gp
        ncv_opt  = ncv

    ncv = ncv + 1

print('        Optimal ncv: ', ncv_opt, "; optimal kernel saved.")

        ncv, rmse_train, rmse_test:  0 8.889007226369564 16.27824570499385
        ncv, rmse_train, rmse_test:  1 9.601809948574594 12.276374494962663
        ncv, rmse_train, rmse_test:  2 9.460692243106418 18.53599211784204
        ncv, rmse_train, rmse_test:  3 7.207371608980688 20.995707038654025
        ncv, rmse_train, rmse_test:  4 10.602249966906736 17.31962807697584
        Optimal ncv:  1 ; optimal kernel saved.


In [5]:
# Come back to the initial training and sets 
X_train_final = X_train
X_test_final  = X_test

# Take the optimal kernel (hyperparameters) to "train" the model on the initial training set 
gp_final      = GaussianProcessRegressor(kernel=opt_gp.kernel_, alpha=0, optimizer=None) #kernel_从cv过程中选择了最优的kernel，The kernel used for prediction. The structure of the kernel is the same as the one passed as parameter but with optimized hyperparameters
gp_final.fit(X_train_final, Y_train)

# Make predictions
y_train_std    = gp_final.predict(X_train_final,return_std=True)
print(np.shape(y_train_std))
y_test_std     = gp_final.predict(X_test_final,return_std=True)
print(np.shape(y_test_std))
y_train = y_train_std[0]
print(np.shape(y_train))
y_test = y_test_std[0]
print(np.shape(y_test))

# Error measures
rmse_train = np.sqrt(mean_squared_error(Y_train, y_train))
rmse_test  = np.sqrt(mean_squared_error(Y_test, y_test))
R2_train_  = gp_final.score(X_train_final, Y_train)
R2_test_   = gp_final.score(X_test_final, Y_test)

# Three optimal hyperparameters can be obtained by the following lines
#print ("k1.k1.constant_value = " + str(gp_final.kernel_.k1.k1.constant_value))
#print ("k2.noise_level       = " + str(gp_final.kernel_.k2.noise_level))
#print ("k2.k2.length_scale   = " + str(gp_final.kernel_.k1.k2.length_scale))

(2, 269)
(2, 30)
(269,)
(30,)


In [6]:
# Visualize the prediction
import matplotlib.pyplot as plt

train_size  = 1.0-test_size
label_train = 'Train: size = ' + str(train_size) +'; R2 = ' + str('%.3f' % R2_train_) + '; rmse = ' + str(
    '%.3f' % rmse_train) #'%.3f' % R2_train_ 小数点后保留三位
label_test  = 'Test:  size = ' + str(test_size) + '; R2 = ' + str('%.3f' % R2_test_) + '; rmse = ' + str(
    '%.3f' % rmse_test)

plt.figure(figsize=(8, 8))
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
#设置图片大小，tick大小

lim_min = min(min(Y_train), min(Y_test), min(y_train), min(y_test))     
lim_max = max(max(Y_train), max(Y_test), max(y_train), max(y_test))
lim = [lim_min - (lim_max - lim_min) * 0.1, lim_max + (lim_max - lim_min) * 0.1] 
plt.xlim(lim)
plt.ylim(lim)

#plt.text(lim_min + (lim_max - lim_min) * 0.4, lim_min + (lim_max - lim_min) * 0.1, label_train)
#plt.text(lim_min + (lim_max - lim_min) * 0.4, lim_min + (lim_max - lim_min) * 0.05, label_test)


plt.xlabel("Computed sublimation enthalpy (kj/mol)", size=17)
plt.ylabel("Predicted sublimation enthalpy (kj/mol)", size=17)


plots_ = list()
plot_train = plt.scatter(Y_train, y_train, marker='o', label="train")
plots_.append(plot_train)
plot_test = plt.scatter(Y_test, y_test, marker='s', label="test")
plots_.append(plot_test)

#errorbar
y_trainerr=y_train_std[1]
y_testerr=y_test_std[1]
plt.errorbar(Y_train, y_train, yerr=y_trainerr,fmt='o')
plt.errorbar(Y_test, y_test, yerr=y_testerr,fmt='o')

plt.legend(handles=[plot_train,plot_test],labels=[label_train,label_test],loc='lower right')

plt.savefig('./sublimation enthalpy_0.1_1.jpg')